In [1]:
from requests import get
import json
import pandas as pd

In [2]:
storkredse = json.load(open("raw_data/TV2/greaterConstituencies.json"))

In [3]:
svar = {}
for storkreds in storkredse:
    print(storkreds['value'], storkreds['label'])
    res = get(f"https://election-api.services.tv2.dk/fv/fv22/candidatetest/answers/{storkreds['value']}")
    svar[storkreds['label']] = res.json()
#gem svarerne
json.dump(svar, open('raw_data/TV2/alle_svar.json', 'w'))


9000400 Bornholms Storkreds
9000600 Fyns Storkreds
9000200 Københavns Omegns Storkreds
9000100 Københavns Storkreds
9001000 Nordjyllands Storkreds
9000300 Nordsjællands Storkreds
9000500 Sjællands Storkreds
9000700 Sydjyllands Storkreds
9000900 Vestjyllands Storkreds
9000800 Østjyllands Storkreds


In [4]:
#svar = json.load(open('raw_data/TV2/alle_svar.json'))

In [5]:
dk_spg = ["1a", "1b", "2a", "2b", "3a", "3b", "4a", "4b", "5a", "5b", "6a", "6b", "7a", "7b", "8a", "8b", "9a", "9b", "10a", "10b", "11a", "11b", "12a", "12b"]

In [6]:
dk_svar = {}
dk_kandidater = {}
for storkreds, kandidater in svar.items():
    for kandidat in kandidater:
        dk_kandidater[kandidat['id']] = {'navn': kandidat['name'], 'storkreds': kandidat['areaName'], 'parti': kandidat['partyName'], 'alder': kandidat['age']}
        dk_svar[kandidat['id']] = {x: kandidat['answers'][x]['answer'] for x in dk_spg if x in kandidat['answers']}

In [7]:
dk_kandidater = pd.DataFrame(dk_kandidater).T
dk_svar = pd.DataFrame(dk_svar).T

In [8]:
q = dk_svar.copy()
q['parti'] = [dk_kandidater.loc[x, 'parti'] for x in dk_svar.index]
q['navn'] = [dk_kandidater.loc[x, 'navn'] for x in dk_svar.index]
q['storkreds'] = [dk_kandidater.loc[x, 'storkreds'] for x in dk_svar.index]
q['alder'] = [dk_kandidater.loc[x, 'alder'] for x in dk_svar.index]
q = q.reset_index()

In [9]:
# udfyld manglende svar med partiets gennemsnit
parti_linjen = q.groupby('parti')[dk_spg].mean()

def fald_ind(x):
    x.loc[x[x.isna()].index] = parti_linjen.loc[x.parti, x[x.isna()].index]
    return x
    
q = q.apply(fald_ind, axis=1)

In [10]:
q.head()

,index,1a,1b,2a,2b,3a,3b,4a,4b,5a,...,10a,10b,11a,11b,12a,12b,parti,navn,storkreds,alder
0,15484,-1,1,0,-1,2,-2,-1,-1,-2,...,1,-1,-1,2,1,-1,Venstre,Julie Pauch Nymark,Bornholms Storkreds,27
1,16235,-2,0,0,1,1,-1,0,1,-1,...,1,1,-2,2,1,1,Venstre,Peter Juel-Jensen,Bornholms Storkreds,56
2,15872,-2,-1,2,2,2,-2,0,-1,-2,...,1,-1,-2,2,2,-2,Dansk Folkeparti,Mette Sode Hansen,Bornholms Storkreds,42
3,15855,-2,-1,1,1,2,-2,1,0,-1,...,2,1,-1,2,0,1,Dansk Folkeparti,Rene Danielsson,Bornholms Storkreds,35
4,15353,-2,0,1,-2,0,-2,2,2,-2,...,2,2,-2,2,1,2,Dansk Folkeparti,Sabine Lyngberg,Bornholms Storkreds,25


In [11]:
q.to_feather("2022_Lasse_data.feather")